<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/webscrap_bedrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ppd_data_jul_aug_2018.csv to ppd_data_jul_aug_2018.csv
User uploaded file "ppd_data_jul_aug_2018.csv" with length 25811766 bytes


In [2]:
import pandas as pd
from pathlib import Path

In [3]:
fields=['unique_id','deed_date','postcode','saon','paon','street','property_type' ]
dfs = [pd.read_csv(csv_file, usecols=fields) for csv_file in uploaded]

In [4]:
results = pd.concat(dfs)
results = results[(results['property_type']!='O')]
display(results)

,unique_id,deed_date,postcode,property_type,saon,paon,street
0,773788C2-A5C2-2CE4-E053-6C04A8C05E57,16/08/2018,AL1 1AT,D,NaN,4,MAGNOLIA GARDENS
1,79A74E21-9532-1289-E053-6B04A8C01627,17/08/2018,AL1 1AU,S,NaN,6,STUART PLACE
2,75050A85-209E-9A88-E053-6B04A8C02390,27/07/2018,AL1 1AU,S,NaN,7,STUART PLACE
3,80E1AA98-1951-7BF8-E053-6C04A8C00BF2,29/08/2018,AL1 1HB,T,NaN,37,HOLYWELL HILL
4,773788C2-A48D-2CE4-E053-6C04A8C05E57,18/07/2018,AL1 1LB,F,FLAT 34,"BENEDICTINE PLACE, 25",LONDON ROAD
...,...,...,...,...,...,...,...
191551,773788C3-7289-2CE4-E053-6C04A8C05E57,15/08/2018,NaN,S,NaN,73,THE COMMON
191554,773788C3-7111-2CE4-E053-6C04A8C05E57,22/08/2018,NaN,D,NaN,3A,STATION WAY
191555,75050A85-F0B9-9A88-E053-6B04A8C02390,20/07/2018,NaN,F,FLAT,2,SWINNOW LANE
191563,75050A85-E882-9A88-E053-6B04A8C02390,01/08/2018,NaN,D,NaN,1,ELCOT NURSERIES


In [5]:
results['rm_url1'] =results['postcode'].str[0:3]+'-' +results['postcode'].str[4:8]+'.html?'
results['rm_url2'] =results['postcode'].str[0:3]+'-' +results['postcode'].str[4:8]+'.html?page=2'   
results['rm_url3'] =results['postcode'].str[0:2]+'-' +results['postcode'].str[3:7]+'.html?'
results['rm_url4'] =results['postcode'].str[0:2]+'-' +results['postcode'].str[3:7]+'.html?page=2'
results['rm_url5'] =results['postcode'].str[0:4]+'-' +results['postcode'].str[5:9]+'.html?'
results['rm_url6'] =results['postcode'].str[0:4]+'-' +results['postcode'].str[5:9]+'.html?page=2'


In [6]:
results_amend1=results.dropna(subset=['postcode'])
#print(results_amend1)

In [7]:
import requests
from bs4 import BeautifulSoup
from re import findall
import re
from pprint import pprint

In [8]:
batchFrom=130001
batchTo=150001
df_1 =results_amend1.iloc[batchFrom:batchTo,:]
#[fromRow:toRow,:] 
#df_1 =results_amend1.iloc[85001:90001,:] 
#df_1 =results_amend1.iloc[165001:,:] #
df_1

,unique_id,deed_date,postcode,property_type,saon,paon,street,rm_url1,rm_url2,rm_url3,rm_url4,rm_url5,rm_url6
137906,75050A85-02AE-9A88-E053-6B04A8C02390,23/07/2018,RM3 0UR,S,NaN,37,REDDEN COURT ROAD,RM3-0UR.html?,RM3-0UR.html?page=2,RM- 0UR.html?,RM- 0UR.html?page=2,RM3 -UR.html?,RM3 -UR.html?page=2
137907,75050A84-FA39-9A88-E053-6B04A8C02390,13/07/2018,RM3 0WJ,F,NaN,26,BENNISON DRIVE,RM3-0WJ.html?,RM3-0WJ.html?page=2,RM- 0WJ.html?,RM- 0WJ.html?page=2,RM3 -WJ.html?,RM3 -WJ.html?page=2
137908,87E1551E-1692-6405-E053-6C04A8C0B2EE,06/07/2018,RM3 0WT,T,NaN,10,JARVIS WAY,RM3-0WT.html?,RM3-0WT.html?page=2,RM- 0WT.html?,RM- 0WT.html?page=2,RM3 -WT.html?,RM3 -WT.html?page=2
137909,773788C2-8175-2CE4-E053-6C04A8C05E57,28/08/2018,RM3 0WW,S,NaN,6,EWAN ROAD,RM3-0WW.html?,RM3-0WW.html?page=2,RM- 0WW.html?,RM- 0WW.html?page=2,RM3 -WW.html?,RM3 -WW.html?page=2
137910,773788C2-801A-2CE4-E053-6C04A8C05E57,31/07/2018,RM3 0XE,T,NaN,41,COPPERFIELDS WAY,RM3-0XE.html?,RM3-0XE.html?page=2,RM- 0XE.html?,RM- 0XE.html?page=2,RM3 -XE.html?,RM3 -XE.html?page=2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159101,75050A85-0537-9A88-E053-6B04A8C02390,07/08/2018,SS14 2NS,T,NaN,180,GREAT SPENDERS,SS1- 2NS.html?,SS1- 2NS.html?page=2,SS-4 2N.html?,SS-4 2N.html?page=2,SS14-2NS.html?,SS14-2NS.html?page=2
159102,75050A85-024A-9A88-E053-6B04A8C02390,31/07/2018,SS14 2PT,T,NaN,28,CURLING TYE,SS1- 2PT.html?,SS1- 2PT.html?page=2,SS-4 2P.html?,SS-4 2P.html?page=2,SS14-2PT.html?,SS14-2PT.html?page=2
159103,726BF13A-AAB2-0A46-E053-6C04A8C01D0D,06/07/2018,SS14 2PT,T,NaN,29,CURLING TYE,SS1- 2PT.html?,SS1- 2PT.html?page=2,SS-4 2P.html?,SS-4 2P.html?page=2,SS14-2PT.html?,SS14-2PT.html?page=2
159104,79A74E21-7F80-1289-E053-6B04A8C01627,20/07/2018,SS14 2PZ,T,NaN,201,CURLING TYE,SS1- 2PZ.html?,SS1- 2PZ.html?page=2,SS-4 2P.html?,SS-4 2P.html?page=2,SS14-2PZ.html?,SS14-2PZ.html?page=2


In [9]:
def findBedrooms(a_url, housenum1, housenum2):
    r = requests.get(a_url)
    c=r.content
    # Parse the html content
    soup = BeautifulSoup(c,"html.parser")
    #print('output from url ' + a_url)
    main_content = soup.find_all('script')
    content_string = str(main_content)
    r1= re.findall(r".address..................|.bedroom.....", content_string)
    #pprint(r1)
    r2=re.findall(r".address..................", content_string)    
    print_bedrooms="no"
    countNoOfFinds=0 
    my_address='not_found'
    my_bedrooms='not_found'
    for eachP in r1:  
      my_check0=eachP.lower().find('bedroom')
      #-----------------------
      if  my_check0==-1: 
        my_check1=eachP.lower().find(housenum1)
      else:
        my_check1=-1
      #-----------------------  
      r_test=re.findall('[0-9]+[a-z]?', eachP)
      if len(r_test)!=0:
         mycheck2=r_test[0] 
      else:
         mycheck2=0
      #-----------------------      
      if (my_check1!=-1 and mycheck2==housenum2 ) or print_bedrooms=="yes" :
          print_bedrooms="yes"  
          if my_check1==-1  :
            countNoOfFinds=countNoOfFinds+1 
            print_bedrooms="no"
            my_bedrooms=eachP
          else:
            my_address=eachP  
            
    return countNoOfFinds, my_address, my_bedrooms        
    print('-------------------------------') 
    print('countNoOfFinds '+ str(countNoOfFinds))
    print('-------------------------------')

In [10]:
def processDF(my_df):
  countIdentifiedRecords=0
  list_for_bedrooms_table=[]
  for row in my_df.itertuples():
    url1='https://www.rightmove.co.uk/house-prices/'+row[8]
    url2='https://www.rightmove.co.uk/house-prices/'+row[9]
    url3='https://www.rightmove.co.uk/house-prices/'+row[10]
    url4='https://www.rightmove.co.uk/house-prices/'+row[11]
    url5='https://www.rightmove.co.uk/house-prices/'+row[12]
    url6='https://www.rightmove.co.uk/house-prices/'+row[13]
    url_list = [url1, url2, url3, url4,url5, url6]
    bedrooms_table_new_row=[]
    propId=row[1] #unique identifier ppd_data
    house_num1=(str(row[5])+" " +str(row[6])).lower()
    if house_num1[0:3]=="nan":
        house_num1=house_num1[4:]       
    find_house_num2= re.findall('[0-9]+[a-z]?', house_num1) 
    if len(find_house_num2)!=0:
      house_num2=find_house_num2[0]
      #print(house_num2) #print 3
    else:
      house_num2 = 0
    bedrooms_table_new_row.append(propId)
    bedrooms_table_new_row.append(house_num1)
    bedrooms_table_new_row.append(house_num2)
    this_prop_num_of_finds=0
    this_prop_num_of_beds=-1
    for each_url in url_list:
      containsSpace = " " in each_url #for cheking if the url string contains spaces
      if not containsSpace:
        if this_prop_num_of_finds==0:
          countNoOfFinds, my_address, my_bedrooms=findBedrooms(each_url,house_num1,house_num2 )
          #print(my_bedrooms)
          countIdentifiedRecords=countIdentifiedRecords + countNoOfFinds
          #----------------------------------------
        else:
          countNoOfFinds=0
          my_address="na" 
          my_bedrooms="na" 
        bedrooms_table_new_row.append(each_url)
        bedrooms_table_new_row.append(countNoOfFinds)        
        bedrooms_table_new_row.append(my_address)
        bedrooms_table_new_row.append(my_bedrooms)
        this_prop_num_of_finds=this_prop_num_of_finds+countNoOfFinds
        if countNoOfFinds>0:
          r_beds=re.findall('[0-9]+', my_bedrooms)
          if len(r_beds)!=0:
            this_prop_num_of_beds=r_beds[0]

    bedrooms_table_new_row.append(this_prop_num_of_finds)
    bedrooms_table_new_row.append(this_prop_num_of_beds)      
    list_for_bedrooms_table.append(bedrooms_table_new_row) 
  #print(list_for_bedrooms_table)
  return list_for_bedrooms_table

In [11]:
def fileOutput(b_from, b_to, a_from, a_to, my_df):
  my_file='batch_{}_{}_bedroom_results_{}_{}.csv'.format(b_from, b_to,a_from, a_to)
  with open(my_file, 'w') as f:
    my_df.to_csv(my_file, index=False)
  files.download(my_file)  

In [12]:
import datetime
my_step=2000
temp_from=0
temp_to=my_step
my_size=batchTo-batchFrom

In [13]:
#
while temp_from<my_size:
  df_temp= df_1.iloc[temp_from:temp_to,:]
  print(str(temp_from) + " _ " + str(temp_to) + ' _ ' +str(datetime.datetime.now()))
  my_bedrooms_list=processDF(df_temp)
  bedroomsDF = pd.DataFrame(my_bedrooms_list)
  bedroomsDF.columns =['id', 'house_num1',  'house_num2','url1', 'num_of_finds1', 'matched_addr1', 'bedrooms1' ,'url2', 'num_of_finds2', 'matched_addr2', 'bedrooms2','overall_num_of_finds','num_of_beds']
  fileOutput(batchFrom, batchTo, temp_from, temp_to, bedroomsDF)
  temp_from=temp_to
  temp_to=temp_from+my_step

0 _ 2000 _ 2021-07-17 01:03:55.990797


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2000 _ 4000 _ 2021-07-17 01:38:52.240438


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

18000 _ 20000 _ 2021-07-17 06:26:16.717084


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New section